In [0]:
import pandas as pd
import os
import tqdm
import csv
import cv2
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import numpy as np

path = "/content/drive/My Drive/APS360 Project/v6-dataset/"

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def countFilesInTest():
    classes = {}
    
    headers = pd.read_csv(os.path.join(path, 'clean-test-id.csv'))

    for j in range(len(headers.filePath)):
        if headers.Class_Description[j] in classes:
            classes[headers.Class_Description[j]] += 1
        else:
            classes[headers.Class_Description[j]] = 1

    return classes

def countFilesInVal():
    classes = {}
    
    headers = pd.read_csv(os.path.join(path, 'clean-validation-id.csv'))

    for j in range(len(headers.filePath)):
        if headers.Class_Description[j] in classes:
            classes[headers.Class_Description[j]] += 1
        else:
            classes[headers.Class_Description[j]] = 1

    return classes

def countFilesInTrain():
    classes = {}
    
    headers = pd.read_csv(os.path.join(path, 'clean-train-id.csv'))

    for j in range(len(headers.filePath)):
        if headers.Class_Description[j] in classes:
            classes[headers.Class_Description[j]] += 1
        else:
            classes[headers.Class_Description[j]] = 1

    return classes

In [0]:
classTrain = countFilesInTrain()
classVal = countFilesInVal()
classTest = countFilesInTest()

# print("Training")
# values = 0
# for key, value in sorted(classTrain.items()):
#     values += value
#     print(f"\t{key}: {value}")
# print(values)

# print("Validation")
# for key, value in sorted(classVal.items()):
#     print(f"\t{key}: {value}")

print("Testing")
for key, value in sorted(classTest.items()):
    print(f"\t{key}: {value}")

Testing
	Apple: 142
	Bagel: 25
	Banana: 66
	Bread: 509
	Broccoli: 114
	Burrito: 51
	Carrot: 60
	Cheese: 238
	Coffee: 313
	Cookie: 287
	Cucumber: 92
	Egg (Food): 190
	French fries: 77
	Grape: 117
	Hamburger: 15
	Hot dog: 23
	Juice: 314
	Lemon: 185
	Lobster: 91
	Muffin: 54
	Orange: 162
	Pancake: 96
	Pasta: 292
	Pear: 46
	Pizza: 42
	Potato: 107
	Salad: 557
	Sandwich: 144
	Strawberry: 202
	Taco: 93
	Tomato: 163
	Waffle: 19


In [0]:
# Read an image from the training set

headers = pd.read_csv(os.path.join(path, 'clean-test-id.csv'))
headers.head()

# Now figure how to write new rows to an existing file
def append_list_as_row(list_of_elem):
    with open(os.path.join(path, 'clean-testing-id.csv'), 'a+', newline='') as write_obj:
        csv_writer = csv.writer(write_obj)
        csv_writer.writerow(list_of_elem)

In [0]:
# Add Gaussian Noise
def addGaussianNoise(image):
    row,col,ch= image.shape
    mean = 0
    var = 0.1
    sigma = var**0.5
    gauss = np.random.normal(mean,sigma,(row,col,ch))
    gauss = gauss.reshape(row,col,ch)
    noisy = image + gauss
    return noisy

In [0]:
# Add flip image on horizontal axis
def flipImageHorizontal(image, XMin, XMax, YMin, YMax):
    # Ymin and Ymax stay the same
    ymin_n = YMin
    ymax_n = YMax

    xmin_n = 1 - XMax
    xmax_n = 1 - XMin
    flipHorizontal = cv2.flip(image, 1)

    return flipHorizontal, xmin_n, xmax_n, ymin_n, ymax_n

# Add flip image on vertical axis
def flipImageVertical(image, XMin, XMax, YMin, YMax):

    ymin_n = 1 - YMax
    ymax_n = 1 - YMin

    # Xmin and Xmax stay the same
    xmin_n = XMin
    xmax_n = XMax

    flipVertical = cv2.flip(image, 0)

    return flipVertical, xmin_n, xmax_n, ymin_n, ymax_n

In [0]:
counter = 0

for i in range(len(headers.filePath)):
    if headers.Class_Description[i] == "Bagel":
        img = cv2.imread(headers.filePath[i])
        newimg, xmin_n, xmax_n, ymin_n, ymax_n = flipImageVertical(img, 
                                                     headers.XMin[i], 
                                                     headers.XMax[i], 
                                                     headers.YMin[i], 
                                                     headers.YMax[i])

        newPath = os.path.join(path, 'train-3', 
                               f'{headers.Class_Description[i]}-VFlip-{i}.png')
        
        cv2.imwrite(newPath, newimg)
        
        append_list_as_row([newPath, headers.Class_Description[i], 
                            xmin_n, xmax_n, 
                            ymin_n, ymax_n])

        counter += 1

In [0]:
# Augment Data using Gaussian Noise

for i in range(len(headers.filePath)):
    if headers.Class_Description[i] == "Bagel":
        img = cv2.imread(headers.filePath[i])
        newimg = addGaussianNoise(img)

        newPath = os.path.join(path, 'train-3', 
                               f'{headers.Class_Description[i]}-GN-{i}.png')
        
        cv2.imwrite(newPath, newimg)
        
        append_list_as_row([newPath, headers.Class_Description[i], 
                            headers.XMin[i], headers.XMax[i], 
                            headers.YMin[i], headers.YMax[i]])


In [0]:
# These ae the classes we will be training in our model

classes = ["Apple",
           "Bagel",
           "Banana",
           "Bread",
           "Broccoli",
           "Burrito",
           "Carrot",
           "Cheese",
           "Coffee",
           "Cookie",
           "Cucumber",
           "Egg (Food)",
           "French fries",
           "Grape",
           "Hamburger",
           "Hot dog",
           "Juice",
           "Lemon",
           "Lobster",
           "Muffin",
           "Orange",
           "Pancake",
           "Pasta",
           "Pear",
           "Pizza",
           "Potato",
           "Salad",
           "Sandwich",
           "Strawberry",
           "Taco",
           "Tomato",
           "Waffle"]

append_list_as_row(["filePath", "Class_Description", "XMin", "XMax", "YMin", "YMax"])

In [0]:
# Clean the csv file

count1 = 0
count2 = 0

classDict = {k: 0 for k in classes}

for i in range(len(headers.filePath)):
    if headers.Class_Description[i] not in classes:
        count1 += 1
        continue
    else:
        count2 += 1
        if classDict[headers.Class_Description[i]] < 1000:
            append_list_as_row([headers.filePath[i], headers.Class_Description[i], 
                                headers.XMin[i], headers.XMax[i], 
                                headers.YMin[i], headers.YMax[i]])
            
            classDict[headers.Class_Description[i]] += 1

print(count1)
print(count2)
print(classDict)

1987
4886
{'Apple': 142, 'Bagel': 25, 'Banana': 66, 'Bread': 509, 'Broccoli': 114, 'Burrito': 51, 'Carrot': 60, 'Cheese': 238, 'Coffee': 313, 'Cookie': 287, 'Cucumber': 92, 'Egg (Food)': 190, 'French fries': 77, 'Grape': 117, 'Hamburger': 15, 'Hot dog': 23, 'Juice': 314, 'Lemon': 185, 'Lobster': 91, 'Muffin': 54, 'Orange': 162, 'Pancake': 96, 'Pasta': 292, 'Pear': 46, 'Pizza': 42, 'Potato': 107, 'Salad': 557, 'Sandwich': 144, 'Strawberry': 202, 'Taco': 93, 'Tomato': 163, 'Waffle': 19}
